In [115]:
import json
import pandas as pd
import glob
from pandas.io.json import json_normalize
pd.options.display.max_rows = 200
pd.options.display.max_columns = 999

## Process the Dirigenti Data

In [117]:
dirigenti_path = 'regione-toscana/atti_dirigenti/dirigenti.json'
out_dirigenti_path = 'regione-toscana/atti_dirigenti/dirigenti_processed.json'

with open(dirigenti_path, 'r') as f:
    dirigenti = json.loads(f.read())
    dirigenti = dirigenti['getDirigenti']['Dirigente']    

with open(out_dirigenti_path, 'w') as f:
    json.dump(dirigenti, f)

df_dirigenti= pd.read_json(out_dirigenti_path)
df_dirigenti.head()

,ID,MATRICOLA,NOME
0,00000000000200320000015922C08452,1352,AGATI STEFANO
1,000000000004002E0000015922C08452,12493,AIAZZI LUCIANO
2,00000000000000050000015922C08452,19514,ALESSANDRO SALVI
3,00000000000200150000015922C08452,1359,ALMANSI GIORGIO
4,00000000000400180000015922C08452,6475,ANCILLI GIUSEPPE


## Process the Office Names

In [118]:
office_path = 'regione-toscana/atti_dirigenti/strutture.json'
out_office_path = 'regione-toscana/atti_dirigenti/strutture_processed.json'

with open(office_path, 'r') as f:
    offices = json.loads(f.read())
    offices = offices['getDg']['Struttura']
    
with open(out_office_path, 'w') as f:
    json.dump(offices, f)
    
df_offices = pd.read_json(out_office_path)
df_offices.head()

,ID,NOME
0,99999,ALTRI UFFICI
1,1923,AVVOCATURA REGIONALE ...
2,50007,D.G. AVVOCATURA ...
3,50073,D.G. COMPETITIVITA' DEL SISTEMA REGIONALE E SV...
4,50079,D.G. DIRITTI DI CITTADINANZA E COESIONE SOCIAL...


## Process the Atti Dirigenti Files

In [72]:
path = 'regione-toscana/atti_dirigenti/regione-toscana*'

In [79]:
files = glob.glob(path)
files[:5]

['regione-toscana/atti_dirigenti/regione-toscana-result-type_atti_dirigenti-year_2016-from_14000-to_14200.json',
 'regione-toscana/atti_dirigenti/regione-toscana-result-type_atti_dirigenti-year_2005-from_2200-to_2400.json',
 'regione-toscana/atti_dirigenti/regione-toscana-result-type_atti_dirigenti-year_2017-from_10600-to_10800.json',
 'regione-toscana/atti_dirigenti/regione-toscana-result-type_atti_dirigenti-year_2017-from_13600-to_13800.json',
 'regione-toscana/atti_dirigenti/regione-toscana-result-type_atti_dirigenti-year_2005-from_5000-to_5200.json']

In [102]:
with open(files[0], 'r') as r:
    data = json.loads(r.read())
    records = data['GSP']['RES']['R']

In [103]:
records[0]

{'CRAWLDATE': '',
 'DU': 'http://www.regione.toscana.it/bancadati/atti/DettaglioAttiD.xml?codprat=2016AD00000017201',
 'ED': '',
 'FS': {'NAME': 'CODICE_PRATICA', 'VALUE': '20160140002016AD00000017201'},
 'HAS': {'C': {'CID': 'NCS_lWDjq4oJ', 'ENC': 'UTF-8', 'SZ': '10k'}, 'L': ''},
 'LANG': 'it',
 'MIME': 'application/pdf',
 'MT': {'ALLEGATO_DESCRIZIONE': ['void'],
  'ALLEGATO_ID': ['0'],
  'ALLEGATO_TIPO_PUBBL': ['void'],
  'CODICE_PRATICA': ['20160140002016AD00000017201'],
  'DATA_ATTO': ['2016-12-19'],
  'DESCRIZIONE': ['Atto pubblicato su BURT e Banca Dati (PBURT/PBD)'],
  'DESCRIZIONE_TIPO_ATTO': ['Decreto non soggetto a controllo interno'],
  'DESCRIZIONE_TIPO_PRATICA': ['Atto dei dirigenti'],
  'DT_UTENTE': ['2016-12-22 11:45:04.83677'],
  'ID_TIPO': ['ADN'],
  'ID_TIPO_PRATICA': ['MON'],
  'LINK_CONTENUTO_BINARIO': ['http://www.regione.toscana.it/bancadati/atti/Contenuto.xml?id=5134532'],
  'NUMERO': ['14000'],
  'OGGETTO': ["DGR 968/07 e s.m.i. Accreditamento degli organismi fo

In [135]:
columns = ['MT.CODICE_PRATICA', 'MT.DATA_ATTO', 'MT.DESCRIZIONE_TIPO_PRATICA', 'MT.ID_TIPO', 'MT.ID_TIPO_PRATICA', 
           'MT.LINK_CONTENUTO_BINARIO', 'MT.OGGETTO', 'MT.PERSONA', 'MT.STATUS', 'MT.UFFICIO_AC',
           'MT.UFFICIO_DG', 'MT.UFFICIO_SE', 'UE']

update_columns = [v[3:] for v in columns[:-1]]
update_columns.append('DETAIL_URL')

In [137]:
update_columns

['CODICE_PRATICA',
 'DATA_ATTO',
 'DESCRIZIONE_TIPO_PRATICA',
 'ID_TIPO',
 'ID_TIPO_PRATICA',
 'LINK_CONTENUTO_BINARIO',
 'OGGETTO',
 'PERSONA',
 'STATUS',
 'UFFICIO_AC',
 'UFFICIO_DG',
 'UFFICIO_SE',
 'DETAIL_URL']

In [138]:
flat_records = json_normalize(records)

df_atti = flat_records[columns]
df_atti.columns = update_columns

def process_values(x):
    if type(x) == list:
        return x[0]
    else:
        return x

df_atti = df_atti.applymap(process_values)

In [139]:
df_atti.head()

,CODICE_PRATICA,DATA_ATTO,DESCRIZIONE_TIPO_PRATICA,ID_TIPO,ID_TIPO_PRATICA,LINK_CONTENUTO_BINARIO,OGGETTO,PERSONA,STATUS,UFFICIO_AC,UFFICIO_DG,UFFICIO_SE,DETAIL_URL
0,20160140002016AD00000017201,2016-12-19,Atto dei dirigenti,ADN,MON,http://www.regione.toscana.it/bancadati/atti/C...,DGR 968/07 e s.m.i. Accreditamento degli organ...,013910,2,00000,50125,05873,http://www.regione.toscana.it/bancadati/atti/D...
1,20160140012016AD00000017466,2016-12-21,Atto dei dirigenti,ADN,MON,http://www.regione.toscana.it/bancadati/atti/C...,REG. CEE 2080/92 - AIUTI ALLE MISURE FORESTALI...,005595,2,00000,50117,05015,http://www.regione.toscana.it/bancadati/atti/D...
2,20160140022016AD00000017198,2016-12-19,Atto dei dirigenti,ADN,MON,http://www.regione.toscana.it/bancadati/atti/C...,"Servizi autorizzati ex art.14 L.R. 42/98 ""Norm...",019075,2,00000,50121,04838,http://www.regione.toscana.it/bancadati/atti/D...
3,20160140032016AD00000017208,2016-12-19,Atto dei dirigenti,ADN,MON,http://www.regione.toscana.it/bancadati/atti/C...,"L.R. 1/2006, DCR 3/2012, L.R. 1/2015 art.29, c...",020415,2,00000,50117,05018,http://www.regione.toscana.it/bancadati/atti/D...
4,20160140042016AD00000017268,2016-12-20,Atto dei dirigenti,ADN,MON,http://www.regione.toscana.it/bancadati/atti/C...,L.R. n. 87/1997 - Pubblicazione sul BURT dell'...,010826,2,00000,50116,05895,http://www.regione.toscana.it/bancadati/atti/D...
